<p><img src="imagenes/rn3.png" width="140" align="left"><img src="http://www.identidadbuho.uson.mx/assets/letragrama-rgb-150.jpg" width="400" align="right"></p>

# [Curso de Redes Neuronales](https://curso-redes-neuronales-unison.github.io/Temario/)

# Predicción en redes neuronales: El algoritmo de *feedforward*

[**Julio Waissman Vilanova**](http://mat.uson.mx/~juliowaissman/), 22 de febrero de 2018.

En esta libreta vamos a practicar con el algoritmo básico para realizar reconocimiento en redes neuronales hacia adelante y establecer una estructura básica para simular cn fines de comprensión. Para aplicaciones reales vamos a utilizar herramientas poderosas como [Tensorflow](https://www.tensorflow.org), pero es importante hacer una primer red neuronal simple a pie con el objetivo de entender mejor los mecanismos básicos.

Como dijo Jack el destripador, vamos por partes, y empecemos con asumir que tenemos la especificación completa de la red neuronal y lo que queremos es poder generar una red neuronal inicial, o poder recuperar una red existente previamente guardada.

Empecemos por inicializar los modulos que vamos a requerir.

In [2]:
import numpy as np
import _pickle as cPickle

## 1. Especificando una red neuronal

Primero, para poder hacer una red neuronal, tenemos que determinar cierta información. La información importante que debemos de especificar cuando hacemos una redes neuronales es:

- Cuantas capas de neuronas tiene la red neuronal, $L$.
- Cuantas neuronas va a tener cada capa $[n_0, n_1, \ldots, n_L]$, donde $n_0$ es el número de entradas y $n_L$ el número de salidas.
- Cual es la función de activación de las neuronas ocultas (logística, lineal rectificada, ...).
- Cual es el tipo de salida de mi red neuronal (lineal, logística, unidad softmax, ... ).
- Los valores con los que se normalizan los datos de entrada a la red neuronal (para el aprendizaje en una red neuronal es muy importante que los valores de entrada estén normalizados).

Una vez que se establecen estos valores, es necesario generar una lista de matrices $[W^{(1)}, \ldots, W^{(L)}]$ donde $W^{(l)}$ es una matriz de dimensiones $(n_l, n_{l-1})$ de pesos. Igualmente es necesario generar una lista de vectores $[b^{(1)}, \ldots, b^{(L)}]$ donde $b^{(l)}$ es un vector de $n_l$ elementos llamados sesgos.

Si se inicializan los valores de las entradas de $W^{(l)}$ y $b^{(l)}$ iguales, es equivalente a tener una sola neurona en esa capa, por lo que es necesario que estos valores sean diferentes. Para este ejemplo y con el fin de simplificar las operaciones de aprendizaje más adelante, vamos a asumir que la función de activación siempre será la función logística.

Para efectos de un mejor aprendizaje, y asumiendo que la función de activación es la logistica, es importante que los valores iniciales de los pesos se encuentren en la zona donde casuan más variacion la función logística. Si asumimos que las entradas a cada neurona están normalizadas (esto es, entre 0 y 1), entonces los pesos deberían ser valores entre $(-\sqrt{n_{l-1}}, \sqrt{n_{l-1}})$ con el fin que la suma se encuentre en la región donde más cambios ocurren en la función logística. 

Vamos a generar y guardar esta información en un diccionario (junto con el resto de la información que requeriramos para tener una red neuronal completamente definida. Al principio los valores de normalización no cuentan ya que estos se deben inicializar al comienzo del aprendizaje. Fijate bien que naturalmente la red neuronal se debería programar como una clase, pero para evitar complejidad que no podamos mantener en TensoFlow vamos a dejarlo todo el código en forma estructurada (solo para dejar constancia).

**Completa el código para inicializar la red neuronal**

In [ ]:
def inicializa_red_neuronal(neuronas_por_capa, tipo):
    """
    Inicializa una red neuronal como un diccionario de datos. 
    Se asume en este caso que la función de activación es la función logística
    
    @param neuronas_por_capa: Una lista de enteros donde el primer elemento es el número de entradas
                              y el último el número de salidas, mientras que los intermedios son
                              el númerode neuronas en cada capa oculta.
    @param tipo: Un string entre {'lineal', 'logistica', 'softmax'} con el tipo de función de salida de la red.
    
    @return: Un diccionario `rn` tal que
             - rn['capas'] = [n0, n1, ..., nL] neuronas por capa
             - rn['tipo'] = tipo
             - rn['W'] = [None, W1, ..., WL] lista de matrices de pesos
             - rn['b'] = [None, b1, ..., bL] lista de sesgos
             - rn['medias'] = lista de medias de cada atributo (se inicializan con puros 0)
             - rn['std'] = lista de desviaciones estandard de cada atributo (se inicializan con puros 1)
             
    """
    rn = {'capas': neuronas_por_capa, 'tipo': tipo}
    rn['medias'] = np.zeros(neuronas_por_capa[0])
    rn['std'] = np.ones(neuronas_por_capa[0])    
    rn['W'] = inicializaW(neuronas_por_capa)
    rn['b'] = inicializab(neuronas_por_capa)
    return rn

def inicializa_W(capas):
    """
    Inicializa una matriz de valores aleatorios W
    
    @param [n0, n1, ..., nL] número de neuronas por capa
    
    @return: Una lista [None, W1, ..., WL] donde cada Wc es un ndarray              
    """
    #------------------------------------------------------------------------
    # Agregua aqui tu código
    return 
    
    #-------------------------------------------------------------------------

def test_inicializa_W():
    #Vamos a hacer 1000 pruebas aleatorias que nos aseguremos que se cumpleen con las especificaciones
    for _ in range(1000):
        n0 = np.random.randint(1, 20)
        n1 = np.random.randint(1, 20)
        W = inicializa_W( n0, n1)
        assert W.shape == (n1, n0 + 1)  # Las dimensiones son correctas
        assert W.max() < np.sqrt(n0)    # La cota máxima se respeta
        assert W.min() > -np.sqrt(n0)   # La cota mínima se respeta
        assert np.abs(W).sum() > 0      # No estamos inicializando a 0
    return "Paso la prueba"

print(test_inicializa_W())
    

Así, si tenemos una red neuronal, la información contenida en el diccionario es toda la información específica que se necesita para la predicción, el aprendizaje, o el reaprendizaje de una red ya especificada. 

Como entrenar una red es algo lento y tedioso, y normalmente cuando hacemos un método de aprendizaje, lo que queremos es poder utilizarlo después para predecir un conjunto de datos no etiquetados previamente, es normal que guardemos en un archivo la información específica a la red neuronal, y despues la recuperemos en otra sesión, otro día, o en otra computadora para hacer la predicción.

Una manera de guardar datos, funciones y objectos de Python en disco es utilizando el módulo ``pickle`` (o su versión compilada para mayor velocidad ``cPickle``). Este modulo permite guardar una serie de objetos de python en forma secuencial en un archivo binario, y luego recuperarlos. Notese que este métdo es diferente a ``np.load`` y ``np.savez``, ya que estos solo permiten guardar (y recuperar) una serie de ndarrays únicamente. 

Vamos entonces a hacer dos funciones muy simples ``guarda_objeto`` y ``carga_objeto``, que utilizaremos más adelante.

In [ ]:
def guarda_objeto(archivo, objeto):
    """
    Guarda un objeto de python en el archivo "archivo". Si el archivo existe, sera reemplazado sin 
    preguntas, al puro estilo mafioso.
    
    @param archivo: string con el nombre de un archivo (aunque no exista)
    @param objeto: Un objeto de python para ser guardado
    
    """
    
    with open(archivo, 'wb') as arch:
        cPickle.dump(objeto, arch, -1)
        arch.close()
        
def carga_objeto(archivo):
    """
    Carga el primer (y se asume que único) objeto contenido en el archivo 'archivo' que debe de ser tipo cPickle.
    
    @param archivo: string con el nombre de un archivo tipo pickle
    @return: El primer objeto dentro del picke
    
    """
    with open(archivo, 'rb') as arch:
        objeto = cPickle.load(arch)
        arch.close()
        return objeto
    
def test_archivo():
    """
    Prueba, para esto vamos a cargar o a leer (o ambas cosas) un objeto en un archivo
    
    Por favor, borrar el archivo cada vez que se pruebe, o para probar la lectura y la escritura
    
    """
    temp = [range(100), 'prueba', True]
    guarda_objeto('prueba.pkl', temp)
    temp =[10, 'no prueba', False]
    otro = carga_objeto('prueba.pkl')
    assert len(otro[0]) == 100
    assert otro[1] == 'prueba'
    assert otro[-1]
    
    return "Pasa la prueba"

print test_archivo()

## 2. Calculando la salida de una red neuronal con *feedforward*


Asumamos que tenemos una red neuronal ya inicializada, y que la vamos a utilizar para calcular el costo de una solución. Como vimos en clase, el costo de la solución depende del tipo de neuronas de salida (que son en realidad la etapa de clasificación). Así, para calcular el costo, es necesario calcular la salida de la red neuronal.

Recordemos que el algoritmo para realizar la alimentación hacia adelante de una red neuronal el algoritmo es el siguiente:

1. Normaliza los valores de entrada

1. Inicializa $a^{(0)}$ asignandole los valores de las entradas normalizadas

2. Por cada capa $l$ de 1 a $L-1$:

    1. Se calcula el valor de $z^{(l)}$ como $$z^{(l)} = W^{(l)} a_e^{(l-1)},$$ donde $W^{(l)}$ es la 
       matriz de pesos de la capa $l-1$ a la capa $l$, y $a_e^{(l-1)}$ es $a^{(l-1)}$ extendida con un 1 al principio 
       el vector.
       
    2. Se calcula $a^{(l)}$ como $$a^{(l)} = g(z^{(l)}),$$ donde $g$ es la función de activación (en nuestro caso hemos 
       decidido utilizar la función logística, pero podríamos tener otras funciones de activación).

3. Se calcula el valor de $z^{(L)}$ como $$z^{(L)} = W^{(L)} a_e^{(L-1)}.$$ 

4. Se calcula $a^{(L)}$ de acuerdo a la función de activación dependiendo del tipo de salida:

    * Si `tipo = 'logistica'` entonces se utiliza la regresión logística (una sola neurona en la capa de salida).
    * Si `tipo = 'lineal'` entonces $a^{(L)} = z^{(L)}$.
    * Si `tipo = 'softmax'` entonces $a^{(L)} = softmax(z^{(L)}).$

5. La salida de la red es $a^{(L)}$.

Aqui hay que tomar en cuenta varias cosas: en primer lugar, la activación de todas las neuronas en todas las capas, y para todos los datos los necesitamos para realizar el algoritmo de *backpropagation*, por lo que se requiere guardarlos. Igualmente, no es eficiente calcular todos los pasos dato por dato, ya que eso lo haría muy, pero muy lento. Así que vamos a aporvechar que los datos vienen en forma de un *ndarray* de numpy y haremos todos los calculos en forma matricial tal como los vimos en clases. 

Sea $X$ la matriz de valores de entrada, entonces $A^{0} = X^T$ es una lista de vectores columna donde cada columna es $a^{(0)}$ para el objeto correspondiente. Así, los calculos se pueden realizar columna por columna, y simplemente
$$ 
Z^{(l)} = W^{(l)}A_e^{(l-1)},
$$
donde $A_e^{(l-1)}$ es $A^{(l-1)}$ agregandole un 1 al inicio de cada vector (o lo que es lo mismo, agregandole un renglon de unos al inicio). Si procedemos de esta forma, entonces es importante recordar que al final $\hat{Y} = (A^{(L)})^T$, ya que para la salida cada renglon es un dato diferente (de acuerdo a nuestra convención desde los otros algoritmos que hamos utilizado), mientras que internamente,para la red neuronal, cada columna proviene de un objeto diferente.

#### Ejercicio 2. Completa el código de las funciones necesarias para realizar el algoritmo de *feedforward* para una red neuronal ya establecida.

In [ ]:
def obtiene_medias_desviaciones(x):
    """
    Obtiene las medias y las desviaciones estandar atributo a atributo.
    
    @param x: un ndarray de dimensión (T, n) donde T es el númro de elementos y n el número de atributos
    @return: medias, desviaciones donde ambos son ndarrays de dimensiones (n,) con las medias y las desviaciones 
             estandar respectivamente.
    
    """
    return x.mean(axis=0), x.std(axis=0)

def normaliza(x, medias, desviaciones):
    """
    Normaliza los datos x

    @param x: un ndarray de dimensión (T, n) donde T es el númro de elementos y n el número de atributos
    @param medias: ndarray de dimensiones (n,) con las medias con las que se normalizará
    @param desviaciones: ndarray de dimensiones (n,) con las desviaciones con las que se normalizará
    
    @return: x_norm un ndarray de las mismas dimensiones de x pero normalizado
    
    """
    return (x - medias) / desviaciones


def logistica(z):
    """
    Calcula la función logística para cada elemento de z
    
    @param z: un ndarray
    @return: un ndarray de las mismas dimensiones que z
    """
    return 1 / (1 + np.exp(-z))

def softmax(z):
    """
    Calculo de la regresión softmax
    
    @param z: ndarray de dimensión (T, K) donde z[i, :] es el vector de aportes lineales de el objeto i    
    @return: un ndarray de dimensión (T, K) donde cada columna es el calculo softmax de su respectivo vector de entrada.
    
    """
    y_hat = np.exp(z)
    return y_hat / y_hat.sum(axis=1).reshape(-1,1)


def extendida(matriz):
    """
    Agrega un renglon de unos a una matriz
    
    """
    return np.r_[np.ones((1, matriz.shape[1])), matriz]

def feedforward(X, red_neuronal):
    """
    Calcula la salida estimada para los valores de `X` utilizando red_neuronal
    
    @param X: ndarray de shape (T, n) donde T es el número de ejemplos y n el número de atributos
    @param red_neuronal: Estructura de datos de una red neuronal inicializada con la función `inicializa_red_neuronal``
    
    @return: `lista_A`, donde `Y_est` es un ndarray de shape (T, k) donde T es el número de objetos y k es 
             el número de salidas (neuronas de salida de la red neuronal).
             
    """    
    # Inicializar A^{(0)}
    # ----------Agragar código aqui -----------------
    
    
    
    
    # Despues vamos a hacer lo propio por cada capa hasta antes de la última
    for wl in red_neuronal['W'][:-1]:         
        # Calcula A_e^{l-1}, Z^{(l)}, A^{l} y agrega a lista_A.        
        # (puede ser todo junto o por partes)
        # ----------Agregar código aqui -----------------

        
        
        lista_A.append(logistica(wl.dot(extendida(lista_A[-1]))))
        
    # Calcula A^{L} y agrega a lista_A de acuerdo al tipo de salida
    # ----------Agregar código aqui -------------------------------
    
    
    
        
    return lista_A
                                      
    
    
def prueba_feedforward():
    """
    Función para validar la función de fedforward 
    
    Se inicializa una red de dos capas (más capa de entrada) y se 
    imponen los pesos. Esto se hizo a mano para las diferentes
    entradas, así que se espera que la función de feedforward de
    cosas similares

    """
    # Inicializa red neuronal
    rn = inicializa_red_neuronal(3, [2, 2, 1], 'lineal')
    
    # Modificamos pesos
    rn['W'][0] = np.array([[0.5, -0.3, -0.7],
                           [-1, 0.2, 0.3]])
    rn['W'][1] = np.array([[0, 0.5, -0.5]])
    
    #Ponemos algunas entradas sencillas
    x = np.array([[0,   0],
                  [1,   1],
                  [-1, -1]])
    
    # Y el valor de A calculado a mano
    A1 = np.array([[0.6225, 0.3775, 0.8176],
                   [0.2689, 0.3775, 0.1824]])
    A2 = np.array([0.17676, 0, 0.3176])
    
    lista_A = feedforward(x, rn)
    assert np.sum(np.abs(lista_A[1] - A1)) <= 0.001
    assert np.sum(np.abs(lista_A[2] - A2)) <= 0.001
    
    #Ahora vamos a ver que pasa si cambiamos la salida
    rn['tipo'] = 'logistica'
    lista_A = feedforward(x, rn)
    A2 = np.array([0.544075, 0.5, 0.5787])

    assert np.sum(np.abs(lista_A[2] - A2)) <= 0.001
    
    return "Paso la prueba"

print(prueba_feedforward())
    

## 3. Calcula una función símple de pérdida de una red neuronal

Con  la función de feedforward desarrollada, podemos hacer una función para calcular la función de pérdida de acuerdo a un conjunto de datos de entrada. La función de pérdida se puede expresar en relación a las salidas $Y$, las salidas estimadas por la red neuronal $\hat{Y}$, y del tipo de salida. 

Las funciones de pérdida más comunes son:

1. Para las redes con salidas lineales el MSE (Mean Square Error) dado por $$Loss(Y, \hat{Y}) = \frac{1}{N}(Y - \hat{Y})^T(Y - \hat{Y}).$$

2. Para redes con salida logística la entropía en dos clases dada por $$Loss(Y, \hat{Y}) = \sum_{i=1}^N{-y^{(i)}\log(\hat{y}^{(i)}) - (1-y^{(i)})\log(1-\hat{y}^{(i)})}$$

3. Para redes con salida *softmax* la entropia multiclase: $$Loss(Y, \hat{Y}) = \sum_{i=1}^N \sum_{k=1}^K{1\{y^{(i)} = k\}\log(\hat{y}^{(i)})}$$

#### Ejercicio 3: Completa el código de la función de perdida de acuerdo a las 3 salidas posibles de la red neuronal.

In [ ]:
def perdida_red(Y, X, rn):
        """
        Calcula la función de perdida de una red neuronal, de acuerdo al tipo de salida
        y a un conjunto de datos conocidos expresados por Y y X

        @param Y: un ndarray de dimension (T, K) con los valores de salida
        @param X: un ndarray de dimension (T, N) con los valores de entrada
        @param rn: una red neuronal'

        """
        lista_A = feedforward(X, rn)
        Y_est = lista_A[-1].T
        return perdida(Y, Y_est, rn['tipo'])



def perdida(Y, Y_est, tipo):
    """
    Calcula la función de perdida de una red neuronal, de acuerdo al tipo de salida 
    
    @param Y: un ndarray de dimension (T, K) con los valores de salida
    @param Y_est: un ndarray de dimension (T, K) con los valores de salida estimados
    @param tipo: Un string que puede ser 'lineal', 'logistica'o 'softmax'
    
    """
    # ----------Agregar código aqui -------------------------------

    
    
    
    
    
def prueba_perdida():
    """
    La unidad de prueba de la función costo
    
    """
    Y = np.array([[.1, 2.2, 3.6, 0]]).T
    Y_est = np.array([[.3, 2.0, 3.8, -0.2]]).T
    assert abs(perdida(Y, Y_est, 'lineal') - 0.02) < 1e-8
    
    Y = np.array([[1, 0, 1, 0]]).T
    Y_est = np.array([[.9, 0.1, 0.8, 0.2]]).T
    assert abs(perdida(Y, Y_est, 'logistica') - 0.164252033486018) < 1e-8
    
    Y = np.array([[1, 0, 1, 0], [0, 1, 0, 0], [0, 0, 0, 1]]).T
    Y_est = np.array([[0.8, 0.1, 0.1], 
                      [0.15, 0.8, 0.05],
                      [0.9, 0.05, 0.05],
                      [0.021, 0.079, 0.9]])
    assert abs(perdida(Y, Y_est, 'softmax') - 0.164252033486018) < 1e-8
    
    return 'paso la prueba'

print prueba_costo()
    